# Hackathon 2: Finetuning DistillBERT

In this notebook we will fine-tune DistillBERT, a transformer based on BERT Googles model to create a toxicity model

You can run this lab both locally or in Colab.

- To run in Colab just go to `https://colab.research.google.com`, sign-in and you upload this notebook. Colab has GPU access for free.
- To run locally just run `jupyter notebook` and access the notebook in this lab. You would need to first install the requirements in `requirements.txt`

You can use any architecture you want! Good luck!

In [ ]:
# CRITICAL: Version constraints for compatibility
# These versions are tested and required for this course
!pip install 'numpy<2' \
             'gensim==4.2.0' \
             'tensorflow==2.15.0' \
             'tensorflow-text==2.15.0' \
             'transformers<4.52' \
             'keras-preprocessing' \
             textblob \
             np_utils

In [ ]:
import os
import random
import warnings

import keras.backend as K
import nltk
import numpy as np
import tensorflow as tf
from textblob import TextBlob

TRACE = False
embedding_dim = 100
rnn_units = 128
epochs=100
buffer_size = 256
max_len = 50
# Batch size
batch_size = 256
min_count_words = 3
BATCH = True

def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  tf.random.set_seed(42)
  random.seed(42)
  if TRACE:
    tf.debugging.set_log_device_placement(True)

set_seeds_and_trace()
warnings.filterwarnings('ignore')
nltk.download('punkt')
textblob_tokenizer = lambda x: TextBlob(x).words

In [ ]:
!bash get_data.sh